<a href="https://colab.research.google.com/github/vijayrgopu/apache_beam/blob/master/4_ParDo_Transform_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install --quiet apache_beam[gcp]

     |████████████████████████████████| 3.4MB 2.9MB/s 
     |████████████████████████████████| 59.2MB 60kB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 1.2MB 34.3MB/s 
     |████████████████████████████████| 225kB 47.0MB/s 
     |████████████████████████████████| 235kB 45.0MB/s 
     |████████████████████████████████| 122kB 52.9MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 143kB 62.0MB/s 
     |████████████████████████████████| 174kB 59.0MB/s 
     |████████████████████████████████| 112kB 61.5MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: google-api-python-client 1.7.12 has requirement httplib2<1dev,>=0.17.0, but you'll have httplib2 0.12.0 which is incompatible.
ERROR: chainer 6.5.0 has requir

In [0]:
from google.colab import files
upload = files.upload()

ERROR! Session/line number was not unique in database. History logging moved to new session 59
Saving dept_data.txt to dept_data.txt


In [0]:
import apache_beam as beam

ModuleNotFoundError: ignored

In [0]:
class FilterAccounts(beam.DoFn):
  def process(self, element):
    if element[3] == 'Accounts':
      return [element]

class PairRecords(beam.DoFn):
  def process(self,element):
    return [(element[3] +" , "+element[1],1)]

class CountRecords(beam.DoFn):
  def process(self,element):
    (key, values) = element
    return [(key, sum(values))]
  
with beam.Pipeline() as p1:
  read_data = (
      p1
      | 'Read From File' >> beam.io.ReadFromText('dept_data.txt')
      | 'Split Rows by Delimiter' >> beam.ParDo(lambda line : [line.split(',')])
      | 'Filter Accounts Data' >> beam.ParDo(FilterAccounts())
      | 'Pair Counts' >> beam.ParDo(PairRecords())
      | 'Group By' >> beam.GroupByKey() #First you will have to group the records before we can count them
      | 'Count Records' >> beam.ParDo(CountRecords())
      | 'Write to TextFile' >> beam.io.WriteToText('data/output') 
  )


In [0]:
!head -20 data/output-00000-of-00001

('Accounts , Marco', 31)
('Accounts , Rebekah', 31)
('Accounts , Itoe', 31)
('Accounts , Edouard', 31)
('Accounts , Kyle', 62)
('Accounts , Kumiko', 31)
('Accounts , Gaston', 31)
('Accounts , Ayumi', 30)
